# About

It's difficult to use functions in jupyter notebook, since we want different steps to be in different cells, so one of the main functions of this module is to emulate a function like scope of the variables - which get destroyed at the end of the experiment. Some extra magic is added to reclaim GPU and General RAM.

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from ipyexperiments import *

## Setup and preload

In [3]:
import numpy as np
import torch

In [4]:
def consume_gpu_ram(n): return torch.ones((n, n)).cuda()
def consume_cpu_ram(n): return np.ones((n, n))

`pytorch`'s CUDA machinery seems to require ~0.5GB GPU RAM, and ~2GB of RAM upon its first use, and it's not shared between processes. So if you use pytorch w/ CUDA your GPU card is 0.5GB smaller from the get going, and multiply that by the number of concurrent processes. 

Because of that, in order to get the numbers right, it can be a good idea to pre-load it by allocating a tiny tensor on `cuda`. If we don't - the first experiment' stats will be misleading/incorrect.

In [5]:
#z = consume_gpu_ram(1)

But, if you use `IPyExperiments` - it performs this preloading for you already, when the backend is loaded (see below).

## Experiment with no GPU

Let's consume a big chunk of non-GPU RAM and reclaim it at the end of the experiment.

In this experiment we use the `cpu` backend, so GPU RAM will not be managed, regardless of whether there is a GPU that can be used or not. This mode is primarily used for configurations without GPU.

In [6]:
exp1 = IPyExperimentsCPU()


*** Experiment started with the CPU-only backend


*** Current state:
RAM:  Used  Free  Total      Util
CPU:   155  3631  31588 MB   0.49% 


･ RAM: △Consumed △Peaked  Used Total | Exec time 0.000s
･ CPU:         0       0      155 MB |


In [7]:
x1 = consume_cpu_ram(2**14) # about 2GB

･ RAM: △Consumed △Peaked  Used Total | Exec time 0.487s
･ CPU:      2048       0     2203 MB |


In [8]:
x2 = consume_cpu_ram(2**14) # about 2GB

･ RAM: △Consumed △Peaked  Used Total | Exec time 1.416s
･ CPU:      2048       0     4160 MB |


In [9]:
del exp1 # finish experiment

･ RAM: △Consumed △Peaked  Used Total | Exec time 0.001s
･ CPU:         0       0     4162 MB |

IPyExperimentsCPU: Finishing

*** Experiment finished in 00:00:02 (elapsed wallclock time)

*** Newly defined local variables:
Deleted: x1, x2

*** Experiment memory:
RAM:  Consumed     Reclaimed
CPU:    4007    4095 MB (102.20%)

*** Current state:
RAM:  Used  Free  Total      Util
CPU:    67  4107  31588 MB   0.21% 




## GPU Experiment: consume general and GPU RAM

Let's consume a big chunk of each, general and GPU RAM and reclaim both of them, at the end of the experiment.

This time we wil use the GPU backed `pytorch`, so both GPU and general RAM will be managed. This is the default backed, so if you don't pass this argument, it'll default to `pytorch`.

In [10]:
exp2 = IPyExperimentsPytorch()


*** Experiment started with the Pytorch backend
Device: ID 0, GeForce GTX 1070 Ti (8119 RAM)


*** Current state:
RAM:  Used  Free  Total      Util
CPU:  2077  2313  31588 MB   6.58% 
GPU:  3689  4430   8119 MB  45.43% 


･ RAM: △Consumed △Peaked  Used Total | Exec time 0.000s
･ CPU:         0       0     2077 MB |
･ GPU:         0       0     3689 MB |


In [11]:
x1 = consume_cpu_ram(2**14) # about 2GB

･ RAM: △Consumed △Peaked  Used Total | Exec time 0.594s
･ CPU:      2048       0     4121 MB |
･ GPU:         0       0     3689 MB |


In [12]:
x2 = consume_gpu_ram(2**14) # about 1GB

･ RAM: △Consumed △Peaked  Used Total | Exec time 2.603s
･ CPU:         0       0     3869 MB |
･ GPU:      1024       0     4713 MB |


In [13]:
del exp2 # finish experiment

･ RAM: △Consumed △Peaked  Used Total | Exec time 0.000s
･ CPU:         0       0     3872 MB |
･ GPU:         0       0     4713 MB |

IPyExperimentsPytorch: Finishing

*** Experiment finished in 00:00:06 (elapsed wallclock time)

*** Newly defined local variables:
Deleted: x1, x2

*** Experiment memory:
RAM:  Consumed     Reclaimed
CPU:    1794    2047 MB (114.08%)
GPU:    1024    1024 MB (100.00%)

*** Current state:
RAM:  Used  Free  Total      Util
CPU:  1825  3121  31588 MB   5.78% 
GPU:  3689  4430   8119 MB  45.43% 




## Get experiment data, preserve some vars

Here we demonstate features that help with using this framework programmatically. i.e. getting the functions to return experiment data during and at the end of the experiment, rather than just printing it. You can then use it to programmatically refine the hyper parameters before rerunning the experiment.

This experiment also demonstrates how to save some of the local variables.

In [14]:
exp3 = IPyExperimentsPytorch()


*** Experiment started with the Pytorch backend
Device: ID 0, GeForce GTX 1070 Ti (8119 RAM)


*** Current state:
RAM:  Used  Free  Total      Util
CPU:  1826  3105  31588 MB   5.78% 
GPU:  3689  4430   8119 MB  45.43% 


･ RAM: △Consumed △Peaked  Used Total | Exec time 0.000s
･ CPU:         0       0     1826 MB |
･ GPU:         0       0     3689 MB |


In [15]:
x1 = consume_cpu_ram(2**14) # about 2GB

･ RAM: △Consumed △Peaked  Used Total | Exec time 0.490s
･ CPU:      2048       0     3875 MB |
･ GPU:         0       0     3689 MB |


Run an intermediary report of how much of the resources was consumed, and how much is available, returning the data as numbers. (none would be reclaimed yet, so it'll be zeros, but the return value is there for consistency).

In [16]:
cpu_data = exp3.data.cpu
gpu_data = exp3.data.gpu
print(cpu_data, gpu_data)

IPyExperimentMemory(consumed=2148294656, reclaimed=0, available=1097605120) IPyExperimentMemory(consumed=0, reclaimed=0, available=4645781504)
･ RAM: △Consumed △Peaked  Used Total | Exec time 0.002s
･ CPU:         0       0     3875 MB |
･ GPU:         0       0     3689 MB |


Let's preserve these variables, so that they remain available after the experiment is finished and the rest of the local variables get deleted. 

Note, that you need to pass the names of the variables and not the variables themselves.

In [17]:
exp3.keep_var_names('cpu_data', 'gpu_data')

･ RAM: △Consumed △Peaked  Used Total | Exec time 0.023s
･ CPU:         0       0     3875 MB |
･ GPU:         0       0     3689 MB |


In [18]:
x2 = consume_gpu_ram(2**14) # about 1GB

･ RAM: △Consumed △Peaked  Used Total | Exec time 0.625s
･ CPU:         0       0     3872 MB |
･ GPU:      1024       0     4713 MB |


Run another intermediary report.

In [25]:
cpu_data = exp3.data.cpu
gpu_data = exp3.data.gpu
print(cpu_data)
print(gpu_data)
print(cpu_data.consumed)
print(gpu_data.available)

IPyExperimentMemory(consumed=-6553600, reclaimed=2151325696, available=3417247744)
IPyExperimentMemory(consumed=0, reclaimed=1073741824, available=4645781504)
-6553600
4645781504


Complete the experiment, delete local vars, reclaim memory, and run the final report of how much of the resources was consumed, and how much is available, and how much was reclaimed, returning the data as numbers.

In [20]:
data = exp3.finish() # finish experiment
# or:
# _ = exp3.finish()
# data = exp3.data
cpu_data_final = data.cpu
gpu_data_final= data.gpu

print("\nNumerical data:\n", cpu_data_final, gpu_data_final)

･ RAM: △Consumed △Peaked  Used Total | Exec time 0.059s
･ CPU:         0       0     3872 MB |
･ GPU:         0       0     4713 MB |

IPyExperimentsPytorch: Finishing

*** Experiment finished in 00:00:01 (elapsed wallclock time)

*** Newly defined local variables:
Deleted: x1, x2
Kept:    cpu_data, gpu_data

*** Experiment memory:
RAM:  Consumed     Reclaimed
CPU:    2045    2047 MB (100.10%)
GPU:    1024    1024 MB (100.00%)

*** Current state:
RAM:  Used  Free  Total      Util
CPU:  1824  3247  31588 MB   5.78% 
GPU:  3689  4430   8119 MB  45.43% 



Numerical data:
 IPyExperimentMemory(consumed=2144772096, reclaimed=2146959360, available=3405377536) IPyExperimentMemory(consumed=1073741824, reclaimed=1073741824, available=4645781504)


And let's test that we can still access the variables we asked not to destroy:

In [21]:
print("\nHalf-way data:\n", cpu_data, gpu_data)


Half-way data:
 IPyExperimentMemory(consumed=2144772096, reclaimed=0, available=1264201728) IPyExperimentMemory(consumed=1073741824, reclaimed=0, available=3572039680)


## Using the context manager

If you want to put all cells into one, you could simplify the experiment even further by using its context manager.

In [22]:
with IPyExperimentsPytorch(): 
    x1 = consume_cpu_ram(2**14)
    x2 = consume_gpu_ram(2**14)


*** Experiment started with the Pytorch backend
Device: ID 0, GeForce GTX 1070 Ti (8119 RAM)


*** Current state:
RAM:  Used  Free  Total      Util
CPU:  1824  3246  31588 MB   5.78% 
GPU:  3689  4430   8119 MB  45.43% 


･ RAM: △Consumed △Peaked  Used Total | Exec time 0.827s
･ CPU:      2048       0     3870 MB |
･ GPU:      1024       0     4713 MB |

IPyExperimentsPytorch: Finishing

*** Experiment finished in 00:00:00 (elapsed wallclock time)

*** Newly defined local variables:
Deleted: x1, x2

*** Experiment memory:
RAM:  Consumed     Reclaimed
CPU:    2045    2048 MB (100.12%)
GPU:    1024    1024 MB (100.00%)

*** Current state:
RAM:  Used  Free  Total      Util
CPU:  1822  3273  31588 MB   5.77% 
GPU:  3689  4430   8119 MB  45.43% 




In [23]:
with IPyExperimentsPytorch() as exp: 
    x1 = consume_cpu_ram(2**14)
    z = "some data"
    x2 = consume_gpu_ram(2**14)
    exp.keep_var_names('z')
print(z)


*** Experiment started with the Pytorch backend
Device: ID 0, GeForce GTX 1070 Ti (8119 RAM)


*** Current state:
RAM:  Used  Free  Total      Util
CPU:  1822  3271  31588 MB   5.77% 
GPU:  3689  4430   8119 MB  45.43% 


･ RAM: △Consumed △Peaked  Used Total | Exec time 0.845s
･ CPU:      2048       0     3868 MB |
･ GPU:      1024       0     4713 MB |

IPyExperimentsPytorch: Finishing

*** Experiment finished in 00:00:00 (elapsed wallclock time)

*** Newly defined local variables:
Deleted: x1, x2
Kept:    z

*** Experiment memory:
RAM:  Consumed     Reclaimed
CPU:    2046    2048 MB (100.08%)
GPU:    1024    1024 MB (100.00%)

*** Current state:
RAM:  Used  Free  Total      Util
CPU:  1820  3262  31588 MB   5.76% 
GPU:  3689  4430   8119 MB  45.43% 


some data


In [24]:
%%javascript # prevent committing an unsaved notebook
IPython.notebook.save_notebook()

<IPython.core.display.Javascript object>